In [71]:
import numpy as np 
X, Y = [], []

def grid_to_tuple(grid):
    tup = []
    for x in grid:
        if x == '-':
            tup.append(0)
        elif x == 'x':
            tup.append(1)
        elif x == 'o':
            tup.append(-1)
        else:
            assert False
    return tuple(tup)

for line in open('train_data.txt').readlines():
    grid, result = line.strip().split()
    X.append(grid_to_tuple(grid))
    Y.append(result == 'True')

X = np.array(X)[:1000]
Y = np.array(Y)[:1000]

In [72]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_small = pca.fit_transform(X)

In [73]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

features = MinMaxScaler(feature_range=(0, np.pi)).fit_transform(X)

train_features, test_features, train_labels, test_labels = train_test_split(
    X, Y, train_size=15, shuffle=False
)

In [74]:
# number of qubits is equal to the number of features
num_qubits = 2

# number of steps performed during the training procedure
tau = 100

# regularization parameter
C = 1000

In [75]:
import qiskit
from qiskit import BasicAer
from qiskit.circuit.library import ZFeatureMap
from qiskit.utils import algorithm_globals

from qiskit_machine_learning.kernels import FidelityQuantumKernel

algorithm_globals.random_seed = 12345

feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=1)

qkernel = FidelityQuantumKernel(feature_map=feature_map)

In [76]:
from qiskit_machine_learning.algorithms import PegasosQSVC

pegasos_qsvc = PegasosQSVC(quantum_kernel=qkernel, C=C, num_steps=tau)

# training
pegasos_qsvc.fit(train_features, train_labels)

# testing
pegasos_score = pegasos_qsvc.score(test_features, test_labels)
print(f"PegasosQSVC classification test score: {pegasos_score}")

PegasosQSVC classification test score: 0.783756345177665
